In [1]:
import os
import json

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm

import transformer_lens

import spacy

In [2]:
# Load the spaCy model
spacy_model = spacy.load("en_core_web_sm")

In [3]:
device_name = "cpu"
if torch.cuda.is_available():
    device_name = "cuda" # CUDA for NVIDIA GPU

device = torch.device(device_name)
print(f"Device: {device_name}")

Device: cuda


In [4]:
model_name = "gpt2-small"
model = transformer_lens.HookedTransformer.from_pretrained(model_name)

Loaded pretrained model gpt2-small into HookedTransformer


In [21]:
text = "This is a test of tokenizers, running jumping Newer, wowrarelongword, Neel Nanda, \"epic\"!"

gpt_tokens = model.to_tokens(text).squeeze(0)
gpt_tokens_str = [model.to_single_str_token(int(t)) for t in gpt_tokens]

print(gpt_tokens)
print(gpt_tokens_str)

tensor([50256,  1212,   318,   257,  1332,   286, 11241, 11341,    11,  2491,
        14284,   968,   263,    11, 28796,   430,  2411,   506,  4775,    11,
         3169,   417,   399,  5282,    11,   366,   538,   291, 40484],
       device='cuda:0')
['<|endoftext|>', 'This', ' is', ' a', ' test', ' of', ' token', 'izers', ',', ' running', ' jumping', ' New', 'er', ',', ' wow', 'ra', 'rel', 'ong', 'word', ',', ' Ne', 'el', ' N', 'anda', ',', ' "', 'ep', 'ic', '"!']


In [27]:
doc = spacy_model(text)
word_idxs = [t.text for t in doc if t.is_alpha]
print(word_idxs)

['This', 'is', 'a', 'test', 'of', 'tokenizers', 'running', 'jumping', 'Newer', 'wowrarelongword', 'Neel', 'Nanda', 'epic']


In [38]:
i = 0
cur = 0 # current word index
sub_idx = 0 # sub index of current word

result = []
while i < len(gpt_tokens_str):
    t = gpt_tokens_str[i].strip()

    cur_word = word_idxs[cur]
    # if token is part of current word, update sub_idx, continue to next token
    if cur_word.find(t, sub_idx) != -1:
        result.append((t, cur))
        sub_idx += len(t)
        i += 1
    else:
        # if token not in cur_word, check next word
        if cur+1 < len(word_idxs) and t in word_idxs[cur+1]:
            cur += 1
            sub_idx = 0
        # if not in cur_word or next word, give up and continue
        else:
            i += 1
            result.append((t, "NA"))

result

[('<|endoftext|>', 'NA'),
 ('This', 0),
 ('is', 1),
 ('a', 2),
 ('test', 3),
 ('of', 4),
 ('token', 5),
 ('izers', 5),
 (',', 'NA'),
 ('running', 6),
 ('jumping', 7),
 ('New', 8),
 ('er', 8),
 (',', 'NA'),
 ('wow', 9),
 ('ra', 9),
 ('rel', 9),
 ('ong', 9),
 ('word', 9),
 (',', 'NA'),
 ('Ne', 10),
 ('el', 10),
 ('N', 11),
 ('anda', 11),
 (',', 'NA'),
 ('"', 'NA'),
 ('ep', 12),
 ('ic', 12),
 ('"!', 'NA')]